In [1]:
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('../')

from infrasim.optimise import *
from infrasim.utils import *

## Run scenarios
Here we run the model across all five scenarios and append the results into a dictionary

In [ ]:
#File paths
nodes = '../data/nextra/spatial/network/nodes.shp'
edges = '../data/nextra/spatial/network/edges.shp'
flows = '../data/nextra/nodal_flows/processed_flows_2030.csv'

# Params
timesteps=None
super_source=False
pprint=True
save_figures=True

infrasim_init_directories()

scenarios = ['BAU','NCO','EAG','COO','UTO']

results = {}
for s in scenarios:
    print('> Running: ' + s)
    model_run = nextra(nodes,edges,flows,
                       scenario='BAU',
                       energy_objective=True,
                       timesteps=timesteps,
                       #super_source=True,
                       #super_sink=True,
                       #res_factor=99,
                       #model_name='meow',
                      )

    model_run.build()
    model_run.run(pprint=False)
    model_run = model_run.get_results()
    # add scenarios to results
    model_run.results_capacities['scenario'] = s
    model_run.results_storages['scenario'] = s
    model_run.results_edge_flows['scenario'] = s
    # append results
    results[s] = model_run
    
print('> Done.')   

> Running: BAU
Academic license - for non-commercial use only - expires 2022-11-03
Using license file /Users/amanmajid/gurobi.lic

> Running: NCO



## Create capacities figure
This figure will show the required capacity (GW) in 2030 per scenario as a bar chart

In [ ]:
def merge_capacity_data(results_dict):
    '''Returns a dataframe with merged capacity data based on a dictionary containing a set of results class
    '''
    # init blank df
    capacities = pd.DataFrame()
    # loop results 
    for k in results_dict.keys():
        df = results_dict[k].results_capacities
        capacities = capacities.append(df,ignore_index=True)
    # map technologies and territories
    capacities = map_tech_and_territory(results['BAU'],capacities,col_to_map='node')
    return capacities

# run function
capacities = merge_capacity_data(results)
# convert to gw
capacities['value'] = capacities['value'].apply(lambda row: mw_to_gw(row))
# pivot for bar chart
capacities = capacities.pivot_table(index=['scenario'],
                                    columns=['technology'],
                                    values='value')
# plot
f,ax = plt.subplots(ncols=1,nrows=1,figsize=(8,5))
capacities.plot.bar(stacked=True,rot=0,ax=ax,cmap='YlGnBu',edgecolor='darkgray')
ax.set_xlabel('')
ax.set_ylabel('Capacity in 2030 [GW]')